In [1]:
from pathlib import Path

import folium
import matplotlib.pyplot as plt
from rasterio.plot import show as rioshow

from hydronetworks import HydroNetworks

In [2]:
import os
import pandas as pd
import rasterio
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import mapping,shape

from shapely.geometry import LineString, MultiLineString


In [4]:

def clip_raster_to_boundary(raster_path, gpkg_path, output_path):
    """
    Clips a raster to the boundary defined in a GeoPackage file.

    Parameters:
    raster_path (str): The path to the input raster file (.tif).
    gpkg_path (str): The path to the GeoPackage file (.gpkg) containing the boundary.
    output_path (str): The path to save the clipped raster file.

    Returns:
    None
    """
    # Load the boundary GeoPackage file
    boundary = gpd.read_file(gpkg_path)

    # Ensure the GeoDataFrame is in the same coordinate reference system (CRS) as the raster
    with rasterio.open(raster_path) as src:
        raster_crs = src.crs
        boundary = boundary.to_crs(raster_crs)

        # Get the geometry as a GeoJSON-like dict
        geometry = [mapping(boundary['geometry'].values[0])]

        # Mask the raster with the boundary
        out_image, out_transform = mask(src, geometry, crop=True)
        out_meta = src.meta.copy()

    # Update the metadata to match the new cropped area
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform
    })

    # Save the clipped raster to a new file
    with rasterio.open(output_path, 'w', **out_meta) as dest:
        dest.write(out_image)

    print(f"Clipping completed. The new file is saved as '{output_path}'.")



In [6]:
## For exluding slope select void-filled DEM (15s resolution)
raster_path = 'data_laos/hyd_as_dem_15s.tif'
gpkg_path = 'data_laos/Archive/Laos.gpkg'
output_path = 'data_laos/dem_laos_void.tif'
clip_raster_to_boundary(raster_path,gpkg_path,output_path)

Clipping completed. The new file is saved as 'data_laos/dem_laos_void.tif'.


In [18]:
raster_path = 'data_laos/dem_con_asia.tif'
gpkg_path = 'data_laos/Laos.gpkg'
output_path = 'data_laos/dem_laos.tif'

clip_raster_to_boundary(raster_path,gpkg_path,output_path)

Clipping completed. The new file is saved as 'data_laos/dem_laos.tif'.


In [19]:
raster_path = 'data_laos/flow_asia.tif'
gpkg_path = 'data_laos/Laos.gpkg'
output_path = 'data_laos/flow_laos.tif'

clip_raster_to_boundary(raster_path,gpkg_path,output_path)

Clipping completed. The new file is saved as 'data_laos/flow_laos.tif'.


In [57]:
raster_path = 'data_laos/as_acc_3s.tif'
gpkg_path = 'data_laos/Laos.gpkg'
output_path = 'data_laos/flowacc_laos.tif'

clip_raster_to_boundary(raster_path,gpkg_path,output_path)

Clipping completed. The new file is saved as 'data_laos/flowacc_laos.tif'.


In [59]:

def clip_and_combine_shapefiles_to_geojson(shapefile_folder, gpkg_path, output_geojson_path):
    """
    Clips all shapefiles in a folder to the boundary defined in a GeoPackage file and saves the combined result as a single GeoJSON file.

    Parameters:
    shapefile_folder (str): The path to the folder containing input shapefiles.
    gpkg_path (str): The path to the GeoPackage file (.gpkg) containing the boundary.
    output_geojson_path (str): The path to save the combined clipped region boundary as a GeoJSON file.

    Returns:
    None
    """
    # Load the boundary from the GeoPackage file
    boundary = gpd.read_file(gpkg_path)

    # Initialize an empty GeoDataFrame with the same CRS as the boundary GeoDataFrame
    combined_gdf = gpd.GeoDataFrame(crs=boundary.crs, geometry=[])

    # Process each shapefile in the folder
    for shapefile in os.listdir(shapefile_folder):
        if shapefile.endswith('.shp'):
            shapefile_path = os.path.join(shapefile_folder, shapefile)

            try:
                # Load the shapefile
                data = gpd.read_file(shapefile_path)

                # Ensure the data is in the same coordinate reference system (CRS) as the boundary
                data = data.to_crs(boundary.crs)

                # Clip the shapefile with the boundary
                clipped = gpd.clip(data, boundary)

                # Append the clipped data to the combined GeoDataFrame
                combined_gdf = pd.concat([combined_gdf, clipped], ignore_index=True)

                print(f"Clipping completed for {shapefile}.")
            except Exception as e:
                print(f"Error processing {shapefile}: {e}")

    # Remove duplicates
    combined_gdf = combined_gdf.drop_duplicates()

    # Ensure the combined GeoDataFrame has an active geometry column
    if 'geometry' in combined_gdf:
        combined_gdf.set_geometry('geometry', inplace=True)

    # Check if the GeoDataFrame is not empty before saving
    if not combined_gdf.empty:
        # Save the combined clipped data as a GeoJSON file
        combined_gdf.to_file(output_geojson_path, driver='GeoJSON')
        print(f"Clipping and combining completed. The new file is saved as '{output_geojson_path}'.")
    else:
        print("No valid geometries found. The combined GeoDataFrame is empty.")


In [60]:
shapefile_path = 'data_laos/HydroRIVERS_v10_as_shp/'
gpkg_path = 'data_laos/Laos.gpkg'
output_path = 'data_laos/rivers.geojson'

In [61]:
clip_and_combine_shapefiles_to_geojson(shapefile_path, gpkg_path, output_path)


Clipping completed for HydroRIVERS_v10_as.shp.
Clipping and combining completed. The new file is saved as 'data_laos/rivers.geojson'.


In [76]:

# Load the original GeoJSON file
input_path = 'data_laos/rivers.geojson'
output_path = 'data_laos/rivers_linestrings.geojson'

# Read the GeoDataFrame
gdf = gpd.read_file(input_path)

# List to hold individual LineString geometries
line_strings = []

# Iterate through each geometry in the GeoDataFrame
for geom in gdf.geometry:
    if isinstance(geom, LineString):
        line_strings.append(geom)
    elif isinstance(geom, MultiLineString):
        for line in geom.geoms:  # Use .geoms to iterate over LineString components
            line_strings.append(line)

# Create a new GeoDataFrame with the LineString geometries
new_gdf = gpd.GeoDataFrame(geometry=line_strings, crs=gdf.crs)

# Save the new GeoDataFrame to a new GeoJSON file
new_gdf.to_file(output_path, driver='GeoJSON')

print(f"Saved {len(line_strings)} LineString geometries to {output_path}")


Saved 24098 LineString geometries to data_laos/rivers_linestrings.geojson


#### Cut-out area around each hydropower plant separately
The idea is to cut out the area, relocate from the selected point to the nearest point at the river and then follow the network creation, as outlined in hydronetworks

In [64]:
from shapely.geometry import Point
from shapely.ops import nearest_points

def nearest_river(point, rivers):
    # Find the nearest river
    nearest_geom = rivers.geometry.unary_union
    nearest_point = nearest_points(point, nearest_geom)[1]
    return nearest_point


In [77]:
hydropower_points = gpd.read_file('data_laos/hydropower_dams.gpkg')
rivers_gdf = gpd.read_file('data_laos/rivers_linestrings.geojson')

In [75]:
# Ensure both GeoDataFrames are in the same CRS
if hydropower_points.crs != rivers_gdf.crs:
    rivers_gdf = rivers_gdf.to_crs(hydropower_points.crs)

# Function to find the nearest point on the LineString
def nearest_river(point, rivers):
    # Find the nearest river
    nearest_geom = rivers.geometry.unary_union
    nearest_point = nearest_points(point, nearest_geom)[1]
    return nearest_point

# Create a new GeoDataFrame to store the relocated points
relocated_points = hydropower_points.copy()

# Apply the nearest river relocation to each point
relocated_points['geometry'] = hydropower_points.apply(lambda row: nearest_river(row.geometry, rivers_gdf), axis=1)

# Save the relocated points to a new file
relocated_points.to_file('data_laos/relocated_points.gpkg', driver='GPKG')

In [84]:
import geopandas as gpd
from shapely.ops import nearest_points
from shapely.geometry import Point

# Paths to the files
hydropower_points_path = 'data_laos/hydropower_dams.gpkg'
rivers_path = 'data_laos/rivers_linestrings.geojson'

# Read the hydropower points from the GPKG file
hydropower_points = gpd.read_file(hydropower_points_path)

# Read the rivers from the GeoJSON file
rivers_gdf = gpd.read_file(rivers_path)

# Ensure both GeoDataFrames are in the same CRS
if hydropower_points.crs != rivers_gdf.crs:
    rivers_gdf = rivers_gdf.to_crs(hydropower_points.crs)

# Verify CRS
print(f"Hydropower points CRS: {hydropower_points.crs}")
print(f"Rivers CRS: {rivers_gdf.crs}")

# Check geometry validity
print("Checking geometry validity...")
if not hydropower_points.is_valid.all():
    hydropower_points['geometry'] = hydropower_points.buffer(0)
if not rivers_gdf.is_valid.all():
    rivers_gdf['geometry'] = rivers_gdf.buffer(0)

# Function to find the nearest point on the LineString
def nearest_river(point, rivers):
    if not isinstance(point, Point):
        raise TypeError("The input point must be a shapely Point object")

    nearest_point = None
    min_distance = float('inf')
    
    # Iterate through each river line
    for line in rivers.geometry:
        if not line.is_empty:
            candidate_point = nearest_points(point, line)[1]
            distance = point.distance(candidate_point)
            if distance < min_distance:
                min_distance = distance
                nearest_point = candidate_point

    if nearest_point is None:
        raise ValueError("No nearest river found")
        
    return nearest_point

# Create a new GeoDataFrame to store the relocated points
relocated_points = hydropower_points.copy()

# Apply the nearest river relocation to each point
relocated_points['geometry'] = hydropower_points.apply(
    lambda row: nearest_river(row.geometry, rivers_gdf) if isinstance(row.geometry, Point) else row.geometry, axis=1
)

# Save the relocated points to a new file
output_path = 'data_laos/relocated_hydropower_dams.gpkg'
relocated_points.to_file(output_path, driver='GPKG')

print(f"Relocation completed and saved to '{output_path}'")


Hydropower points CRS: EPSG:4326
Rivers CRS: EPSG:4326
Checking geometry validity...
Relocation completed and saved to 'data_laos/relocated_hydropower_dams.gpkg'


In [87]:
import geopandas as gpd
from shapely.geometry import Point
from scipy.spatial import KDTree
import numpy as np

# Read the hydropower points and rivers
hydropower_points = gpd.read_file('data_laos/hydropower_dams.gpkg')
rivers_gdf = gpd.read_file('data_laos/rivers_linestrings.geojson')

# Ensure both GeoDataFrames are in the same CRS
if hydropower_points.crs != rivers_gdf.crs:
    rivers_gdf = rivers_gdf.to_crs(hydropower_points.crs)

# Extract coordinates of river lines
river_coords = np.vstack(rivers_gdf.geometry.apply(lambda geom: np.array(geom.coords).tolist()).values)
river_tree = KDTree(river_coords)

# Function to find the nearest point using KDTree
def nearest_river_kdtree(point):
    dist, idx = river_tree.query(point.coords[0])
    nearest_point = Point(river_coords[idx])
    return nearest_point

# Apply the KDTree nearest point calculation
hydropower_points['geometry'] = hydropower_points.apply(
    lambda row: nearest_river_kdtree(row.geometry) if isinstance(row.geometry, Point) else row.geometry, axis=1
)

# Save the relocated points to a new file
hydropower_points.to_file('data_laos/relocated_hydropower_dams_kdtree.gpkg', driver='GPKG')


In [89]:
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points

# Read the hydropower points and rivers
hydropower_points = gpd.read_file('data_laos/hydropower_dams.gpkg')
rivers_gdf = gpd.read_file('data_laos/rivers_linestrings.geojson')

# Ensure both GeoDataFrames are in the same CRS
if hydropower_points.crs != rivers_gdf.crs:
    rivers_gdf = rivers_gdf.to_crs(hydropower_points.crs)

# Function to find the nearest point on the LineString with optimization
def nearest_river_iterative(point, rivers):
    if not isinstance(point, Point):
        raise TypeError("The input point must be a shapely Point object")
    
    nearest_point = None
    min_distance = float('inf')
    
    for line in rivers.geometry:
        if not line.is_empty:
            candidate_point = nearest_points(point, line)[1]
            distance = point.distance(candidate_point)
            if distance < min_distance:
                min_distance = distance
                nearest_point = candidate_point

    if nearest_point is None:
        raise ValueError("No nearest river found")
        
    return nearest_point

# Apply the iterative nearest point calculation
hydropower_points['geometry'] = hydropower_points.apply(
    lambda row: nearest_river_iterative(row.geometry, rivers_gdf) if isinstance(row.geometry, Point) else row.geometry, axis=1
)

# Save the relocated points to a new file
hydropower_points.to_file('data_laos/relocated_hydropower_dams_iterative.gpkg', driver='GPKG')


In [92]:
import geopandas as gpd
from shapely.validation import explain_validity

# Read the hydropower points and rivers
hydropower_points = gpd.read_file('data_laos/hydropower_dams.gpkg')
rivers_gdf = gpd.read_file('data_laos/rivers_linestrings.geojson')

# Ensure both GeoDataFrames are in the same CRS
if hydropower_points.crs != rivers_gdf.crs:
    rivers_gdf = rivers_gdf.to_crs(hydropower_points.crs)

# Check for invalid geometries
print("Checking for invalid geometries in hydropower points...")
invalid_points = hydropower_points[~hydropower_points.is_valid]
for idx, row in invalid_points.iterrows():
    print(f"Invalid Point {idx}: {explain_validity(row.geometry)}")

print("Checking for invalid geometries in rivers...")
invalid_rivers = rivers_gdf[~rivers_gdf.is_valid]
for idx, row in invalid_rivers.iterrows():
    print(f"Invalid River {idx}: {explain_validity(row.geometry)}")

# Fix invalid geometries
hydropower_points['geometry'] = hydropower_points.apply(
    lambda row: row.geometry.buffer(0) if not row.geometry.is_valid else row.geometry, axis=1)
rivers_gdf['geometry'] = rivers_gdf.apply(
    lambda row: row.geometry.buffer(0) if not row.geometry.is_valid else row.geometry, axis=1)


Checking for invalid geometries in hydropower points...
Checking for invalid geometries in rivers...
